In [1]:
%load_ext autoreload
%autoreload 2

# A/B testing scratch

A place to check your tests will work. An unsacred space for development.

In [5]:
import logging
import os
from http import HTTPStatus
from typing import Annotated
from uuid import UUID

from fastapi import Depends, FastAPI, HTTPException, WebSocket
from fastapi.encoders import jsonable_encoder
from langchain.chains.llm import LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_community.chat_models import ChatLiteLLM
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_elasticsearch import ApproxRetrievalStrategy, ElasticsearchStore
from elasticsearch import Elasticsearch

from core_api.src.auth import get_user_uuid

from redbox.model_db import MODEL_PATH
from redbox.models import EmbeddingModelInfo, Settings
from redbox.models.chat import ChatRequest, ChatResponse, SourceDocument
from redbox.models.settings import ElasticLocalSettings
from redbox.storage import ElasticsearchStorageHandler

env = Settings(
    _env_file="../.env",
    minio_host="localhost",
    elastic=ElasticLocalSettings(
        host = "localhost"
    )
)

embedding_model = SentenceTransformerEmbeddings(model_name=env.embedding_model, cache_folder="../models/")

es = Elasticsearch(
    hosts=[
        {
            "host": "localhost",
            "port": env.elastic.port,
            "scheme": env.elastic.scheme,
        }
    ],
    basic_auth=(env.elastic.user, env.elastic.password),
)

if env.elastic.subscription_level == "basic":
    strategy = ApproxRetrievalStrategy(hybrid=False)
elif env.elastic.subscription_level in ["platinum", "enterprise"]:
    strategy = ApproxRetrievalStrategy(hybrid=True)

vector_store = ElasticsearchStore(
    es_connection=es,
    index_name="redbox-data-chunk",
    embedding=embedding_model,
    strategy=strategy,
    vector_query_field="embedding",
)

llm = ChatLiteLLM(
    model=env.azure_openai_model,
    streaming=True,
    azure_key=env.azure_openai_api_key,
    api_version=env.openai_api_version,
    api_base=env.azure_openai_endpoint,
    max_tokens=4_096,
)

storage_handler = ElasticsearchStorageHandler(es_client=es, root_index=env.elastic_root_index)

/Users/willlangdale/Library/Caches/pypoetry/virtualenvs/redbox-Vh_-Fb0j-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/willlangdale/Library/Caches/pypoetry/virtualenvs/redbox-Vh_-Fb0j-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:sentence_transform

## K-retrieval

In [22]:
from core_api.src.build_chains import build_k_retrieval_chain
from core_api.src.runnables import make_es_retriever

In [20]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {
            "terms": {
                "parent_file_uuid.keyword": [
                    "7bcc6d44-6bf3-4c45-b598-8f421531daa2",
                    "a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f"
                ]
            }
        }
        # "filter": {
        #     "parent_file_uuid": "7bcc6d44-6bf3-4c45-b598-8f421531daa2"
        # }
    }
)

retriever.invoke("yest")

INFO:elastic_transport.transport:POST http://localhost:9200/redbox-data-chunk/_search?_source_includes=metadata,text [status:200 duration:0.012s]


[Document(page_content='Buildings', metadata={'parent_doc_uuid': '7bcc6d44-6bf3-4c45-b598-8f421531daa2', 'languages': ['eng'], 'link_texts': None, 'link_urls': None, 'links': None, 'page_number': 4}),
 Document(page_content='4. "Cabinet Ofﬁce, About Us" (https://www.gov.uk/government/organisations/cabinet-ofﬁce/about). HM Government. Retrieved 29 March 2020.\n\n5. Government Commercial Function: Looking to the Future (https://assets.publishing.service.go v.uk/government/uploads/system/uploads/attachment_data/ﬁle/711988/Rich_Picture.jpg), accessed 5 May 2019', metadata={'parent_doc_uuid': '7bcc6d44-6bf3-4c45-b598-8f421531daa2', 'languages': ['eng'], 'link_texts': None, 'link_urls': None, 'links': None, 'page_number': 5}),
 Document(page_content='Service; Political and constitutional reform', metadata={'parent_doc_uuid': '7bcc6d44-6bf3-4c45-b598-8f421531daa2', 'languages': ['eng'], 'link_texts': None, 'link_urls': None, 'links': None, 'page_number': 1}),
 Document(page_content='and we we

In [106]:
from langchain_elasticsearch import ElasticsearchRetriever
from langchain_core.runnables import ConfigurableField
from typing import Any, TypedDict, Callable
from redbox.models import Chunk
from langchain_core.retrievers import BaseRetriever
from functools import partial


class ESQuery(TypedDict):
    question: str
    file_uuids: list[UUID]
    user_uuid: UUID


class ESParams(TypedDict):
    size: int
    num_candidates: int
    match_boost: float
    knn_boost: float
    similarity_threshold: float


def get_es_retriever(
    env, es
) -> BaseRetriever:
    """Creates an Elasticsearch retriever runnable.

    Runnable takes input of a dict keyed to question, file_uuids and user_uuid.

    Runnable returns a list of Chunks.
    """

    def es_query(query: ESQuery, params: ESParams) -> dict[str, Any]:
        vector = embedding_model.embed_query(query["question"])

        query_filter = [{"term": {"creator_user_uuid.keyword": str(query["user_uuid"])}}]

        if len(query["file_uuids"]) != 0:
            query_filter.append({"terms": {"parent_file_uuid.keyword": [str(uuid) for uuid in query["file_uuids"]]}})

        return {
            "size": params["size"],
            "query": {
                "bool": {
                    "should": [
                        {
                            "match": {
                                "text": {
                                    "query": query["question"],
                                    "boost": params["match_boost"],
                                }
                            }
                        },
                        {
                            "knn": {
                                "field": "embedding",
                                "query_vector": vector,
                                "num_candidates": params["num_candidates"],
                                "filter": query_filter,
                                "boost": params["knn_boost"],
                                "similarity": params["similarity_threshold"],
                            }
                        }
                    ],
                    "filter": query_filter
                }
            },
        }

    def chunk_mapper(hit: dict[str, Any]) -> Chunk:
        return Chunk(**hit["_source"])
    
    class ParameterisedElasticsearchRetriever(ElasticsearchRetriever):
        params: ESParams
        body_func: Callable[[str], dict]

        def __init__(self, **kwargs: Any) -> None:
            super().__init__(**kwargs)
            self.body_func = partial(self.body_func, params=self.params)

    default_params = {
        "size": env.ai.rag_k,
        "num_candidates": env.ai.rag_num_candidates,
        "match_boost": 1,
        "knn_boost": 1,
        "similarity_threshold": 0
    }

    return ParameterisedElasticsearchRetriever(
        es_client=es, 
        index_name=f"{env.elastic_root_index}-chunk", 
        body_func=es_query, 
        document_mapper=chunk_mapper,
        params=default_params
    ).configurable_fields(
        params=ConfigurableField(
            id="params",
            name="Retriever parameters",
            description="A dictionary of parameters to use for the retriever."
        )
    )

chat_request = ChatRequest(
    **{
        "message_history": [
            {
                "text": "Tell me about energy", "role": "user"
            },
        ],
        "selected_files": [ 
            {"uuid": "718dfb9c-3f0c-4942-a0c1-e0458a7a53c6"},
            {"uuid": "a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f"}
        ]
    }
)

retriever = get_es_retriever(
    env=env,
    es=es
)

response = (
    retriever
    .with_config(
        configurable={
            "params": {
                "size": 10,
                "num_candidates": 100,
                "match_boost": 1,
                "knn_boost": 2,
                "similarity_threshold": .7,
            }
        }
    )
    .invoke({
        "question": "BEIS statistical publications Energy Trends",
        "file_uuids": [
            "718dfb9c-3f0c-4942-a0c1-e0458a7a53c6",
            "a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f"
        ],
        "user_uuid": "b92ebddb-a77e-4ed7-81b9-a2f7ce814ef5"
    })
)

[
    (res.parent_file_uuid, res.text)
    for res in response
]

# len(response["sources"]), response["reponse"]

INFO:elastic_transport.transport:POST http://localhost:9200/redbox-data-chunk/_search [status:200 duration:0.009s]


[(UUID('a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f'),
  'Also published today are the September editions of the BEIS statistical publications Energy Trends and Energy Prices which contain data for Q2 2022 (April to June). The main points to note are:\n\nEnergy Trends'),
 (UUID('a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f'),
  'Simon, Stuart & Anouka cc: PS/BEIS Ministers, SPADs, Perm Sec, DGs & CSA\n\nPlease find attached the latest brief on monthly energy data which are published today at: https://www.gov.uk/government/statistics/energy-trends-and-prices-statistical- release-29-september-2022\n\nHighlights for the 3 month period May to July 2022 compared to the same period a year earlier are:'),
 (UUID('718dfb9c-3f0c-4942-a0c1-e0458a7a53c6'),
  'BEIS - Monthly energy statistics briefing note\n\nSeptember 2022'),
 (UUID('a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f'),
  '---------- Forwarded message --------- From: Harris, Kevin (TIUA - Analysis Directorate) <Kevin.Harris@beis.gov.uk> Date: Thu, 29 Sept 20

In [23]:
chat_request = ChatRequest(
    **{
        "message_history": [
            {
                "text": "Tell me about energy", "role": "user"
            },
        ],
        "selected_files": [ 
            {"uuid": "718dfb9c-3f0c-4942-a0c1-e0458a7a53c6"},
            {"uuid": "a28c04e2-8a1c-41b0-8d29-74ae41aa2e0f"}
        ]
    }
)

chain, params = await build_k_retrieval_chain(
    chat_request=chat_request,
    user_uuid=UUID("b92ebddb-a77e-4ed7-81b9-a2f7ce814ef5"),
    llm=llm,
    embedding_model=embedding_model,
    storage_handler=storage_handler,
    k=20
)

response = chain.invoke(params)
len(response["sources"]), response["reponse"]

INFO:elastic_transport.transport:POST http://localhost:9200/redbox-data-chunk/_search [status:200 duration:0.012s]
INFO:elastic_transport.transport:POST http://localhost:9200/redbox-data-chunk/_search [status:200 duration:0.011s]
15:50:33 - LiteLLM:INFO: utils.py:1307 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://oai-i-dot-ai-playground-sweden.openai.azure.com//openai/deployments/gpt-4/ \
-H 'Authorization: Bearer a87792838865********************' \
-d '{'model': 'gpt-4', 'messages': [{'role': 'system', 'content': "You are Redbox. An AI focused on helping UK Civil Servants, Political Advisors and Ministers triage and summarise information from a wide variety of sources. You are impartial and non-partisan. You are not a replacement for human judgement, but you can help humans make more informed decisions. If you are asked a question you cannot answer based on your following instructions, you should say so. Be concise and professional in your responses. Respond in markdown f

{'response': 'Based on the extracted documents, here are some key points on energy:\n\n- **Primary energy consumption in the UK** has increased by 2.3%, with petrol consumption rising due to the easing of lockdown restrictions. When adjusted for temperature, consumption rose by 3.9%.\n\n- Indigenous energy production experienced a significant boost, rising by 21%. This surge was largely driven by strong growth in UKCS production.\n\n- The UK has played a pivotal role in **supplying gas to Europe** as it seeks to reduce its reliance on Russian gas, which has led to a notable increase in gas exports.',
 'sources': [Chunk(uuid=UUID('e28769db-ab4d-4c0b-8b52-28fd5f4320ee'), created_datetime=datetime.datetime(2024, 6, 11, 4, 17, 29, 83840), creator_user_uuid=UUID('b92ebddb-a77e-4ed7-81b9-a2f7ce814ef5'), parent_file_uuid=UUID('718dfb9c-3f0c-4942-a0c1-e0458a7a53c6'), index=0, text='BEIS - Monthly energy statistics briefing note\n\nSeptember 2022', metadata=Metadata(parent_doc_uuid=UUID('718dfb